# Bilinear upsampling with Deconvolution layer

In [1]:
using MXNet
using Images

### Calculate the appropriate kernel size for scaling factor

In [2]:
factor = 3
kernel = 2factor - factor % 2
stride = factor
pad = ceil(Int64, (factor - 1) / 2);

### Create network

In [3]:
data = mx.Variable(:data)
deconv = mx.Deconvolution(data = data, num_filter = 1,
                          kernel = (kernel, kernel),
                          stride = (stride, stride),
                          pad = (pad, pad), no_bias=true)
deconv_w =  mx.list_arguments(deconv)[2]

:deconvolution0_weight

### Create test image

In [4]:
data = zeros(3, 3, 1, 1) # width, height, channels, samples
data[:, :, 1, 1] = [[0 0.25 0.5]; [0.25 0.5 0.75]; [0.5 0.75 1]]
original = grayim(data[:, :, 1, 1])

### Initialize network

In [5]:
dp = mx.ArrayDataProvider(data, batch_size = 1)
model = mx.FeedForward(deconv)
mx.init_model(model, Dict(:default => mx.UniformInitializer(), deconv_w => mx.BilinearInitializer()), data=size(data))

([:data,:deconvolution0_weight],Any[:deconvolution0_weight],Symbol[])

### Run network over data

In [6]:
pred = mx.predict(model, dp)
grayim(pred[:, :, 1, 1])